In [1]:
import os
import csv
import json
import pandas as pd
import numpy as np
import tensorflow as tf

from math import ceil
from joblib import Parallel, delayed

import multiprocessing
from tqdm.notebook import tqdm

from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs

2023-03-28 14:18:26.907953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 14:18:27.616852: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:/usr/local/cuda-11.6/lib64:
2023-03-28 14:18:27.616960: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:/usr/local/cuda-11.6/lib64:
2023-03-28 1

In [2]:
from tqdm.notebook import tqdm

In [3]:
tqdm.pandas()

In [4]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large",
    "embeddings":"music_style"
})


In [5]:

# In[16]:


base_path = os.path.join(args.root_dir,"fma")


# In[17]:


models_path = os.path.join(args.root_dir,"models")


metadata_path_fma = os.path.join(base_path,"fma_metadata")



In [6]:

if args.embeddings == "music_style":
    model_path = os.path.join(models_path,args.embeddings,"discogs-effnet-bs64-1.pb")



In [7]:
df = pd.read_csv(os.path.join(metadata_path_fma,"tracks_valid.csv"))

In [8]:
model = TensorflowPredictEffnetDiscogs(graphFilename=model_path,output="PartitionedCall:1")

2023-03-28 14:18:45.513457: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.65GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2023-03-28 14:18:45.513942: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-03-28 14:18:45.513975: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-03-28 14:18:45.513984: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2023-03-28 14:18:45.513991: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2023-03-28 14:18:45.514447: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1418] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8322 MB memory) -> physical GPU (device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:6

In [9]:

def create_dir(path):
    # checking if the directory demo_folder2 
    # exist or not.
    if not os.path.isdir(path):

        # if the demo_folder2 directory is 
        # not present then create it.
        os.makedirs(path)
    return True


def extract_feature(file_path,model):
    ### Configuração do model para extrair a representação do aúdio
    # model = TensorflowPredictEffnetDiscogs(graphFilename=model_path)
    
    
    audio = MonoLoader(filename=file_path, sampleRate=16000)()
    activations = model(audio)
    
    
    return activations


In [10]:
def find_path(track_id,dataset_path):
    track_id = track_id.zfill(6)
    folder_id = track_id[0:3]
    file_path = os.path.join(dataset_path,folder_id,track_id+'.mp3')
    return file_path
    


# In[36]:


df['file_path'] = df.track_id.apply(lambda x: find_path(str(x),args.dataset_path))



In [11]:

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array


def parse_single_music(data,music):
    # cat1, cat2, cat3, cat4, cat5 = data
    track_id, track_title, valid_genre, file_path = data
    
    #define the dictionary -- the structure -- of our single example
    data = {
        'emb' : _bytes_feature(serialize_array(music)),
        'track_id' : _int64_feature(track_id)
    }
    #create an Example, wrapping the single features
    out = tf.train.Example(features=tf.train.Features(feature=data))

    return out
# In[45]:




In [13]:


# In[24]:

def process_df(df,i,count,batch_size,tfrecords_path):
    batch_df = df[i:i+batch_size]
        
    tqdm.pandas()

    X = batch_df.file_path.progress_apply(lambda x: extract_feature(x,model))   

    print("Extraiu as features")

    batch_df = batch_df[['first_genre_id_label']]


    tfrecords = [parse_single_music(data, x,labels) for data, x in zip(batch_df.values, X)]

    path = os.path.join(tfrecords_path,f"{str(count).zfill(10)}.tfrecord")

    #with tf.python_io.TFRecordWriter(path) as writer:
    with tf.io.TFRecordWriter(path) as writer:
        for tfrecord in tfrecords:
            writer.write(tfrecord.SerializeToString())

    print(f"{count} {len(tfrecords)} {path}")


def generate_tf_records(df,model,filename="train"):
    
    tfrecords_path = os.path.join(args.dataset_path,"tfrecords",filename)
    
    create_dir(tfrecords_path)
    
    
    batch_size = 1024 * 10  # 10k records from each file batch
    count = 0
    total = ceil(len(df) / batch_size)
    
    with Parallel(n_jobs=20, require='sharedmem') as para:
        print("Estamos usando paralelismo!!!")
        para(delayed(process_df)(df,i,count,batch_size,tfrecords_path) for count,i in enumerate(range(0, len(df), batch_size)))
    

# In[31]:


In [14]:
df

,track_id,track_title,valid_genre,file_path
0,2,Food,['21'],/mnt/disks/data/fma/fma_large/000/000002.mp3
1,3,Electric Ave,['21'],/mnt/disks/data/fma/fma_large/000/000003.mp3
2,5,This World,['21'],/mnt/disks/data/fma/fma_large/000/000005.mp3
3,10,Freeway,['10'],/mnt/disks/data/fma/fma_large/000/000010.mp3
4,20,Spiritual Level,"['76', '103']",/mnt/disks/data/fma/fma_large/000/000020.mp3
...,...,...,...,...
104181,155316,The Auger,['25'],/mnt/disks/data/fma/fma_large/155/155316.mp3
104182,155317,Let's Skin Ruby,['25'],/mnt/disks/data/fma/fma_large/155/155317.mp3
104183,155318,My House Smells Like Kim Deal/Pulp,['25'],/mnt/disks/data/fma/fma_large/155/155318.mp3
104184,155319,The Man With Two Mouths,['25'],/mnt/disks/data/fma/fma_large/155/155319.mp3


In [ ]:
generate_tf_records(df,model,filename=args.embeddings)

Estamos usando paralelismo!!!


  0%|          | 0/10240 [00:00<?, ?it/s]